In [ ]:
import pandas as pd

In [ ]:
csv_file = "../data/tweet_activity_metrics_tkosht_20240106_20240203_ja.csv"
df = pd.read_csv(csv_file)
df.head(3)

In [ ]:
from urllib.robotparser import RobotFileParser
from urllib.parse import urlparse

url = "https://twitter.com/ai_database/status/1756147762822058066"
print(f"{url=}")
parsed = urlparse(url)

url_robots = f"{parsed.scheme}://{parsed.netloc}/robots.txt"
print(f"{url_robots=}")

user_agent = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/121.0.6167.85 Safari/537.36"
rbp = RobotFileParser(url=url_robots)
rbp.read()
rbp.can_fetch(user_agent, url)

In [ ]:
entry = rbp.entries[0]
rule = entry.rulelines[0]
rule.allowance

In [ ]:
import os


os.environ["PATH"].split(":")[-5:]

In [ ]:
# Chrome
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

# from webdriver_manager.chrome import ChromeDriverManager


def create_driver():
    options = Options()
    options.binary_location = "/opt/chrome-linux64/chrome"
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--enable-chrome-browser-cloud-management")
    options.add_argument("--enable-javascript")
    service = Service("/usr/local/bin/chromedriver")
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def fetch_text_with_selenium(url: str, finds: list = [], max_tries: int = 3):
    driver = create_driver()

    max_wait = 13
    text = None
    try:
        for n_try in range(max_tries):
            try:
                driver.get(url)

                # wait appearance
                for finder in finds:
                    # eg) By.XPATH, "//div", "copyright" = rec : (str, str, str)
                    by, by_elm, finder_text = finder

                    try:
                        WebDriverWait(driver, max_wait).until(
                            expected_conditions.text_to_be_present_in_element(
                                (by, by_elm), text_=finder_text
                            )
                        )
                        text = driver.find_element(by=by, value=by_elm).text
                        break
                    except TimeoutException:
                        continue

                if not text:
                    # browser.implicitly_wait(10)
                    text = driver.find_element(by=By.TAG_NAME, value="body").text
                return text
            except TimeoutException as te:
                if n_try > max_tries:
                    print(f"[WARNING] fetching text was failed. [{url=}]")
                    raise te
                t = 2**n_try
                print(f"sleeping to retry ... [{t=} secs]")
                time.sleep(t)
    finally:
        driver.quit()

In [ ]:
# driver = create_driver()
# driver.execute_script("return navigator.userAgent;")
# driver.quit()

In [ ]:
# Firefox
# # NOTE: https://www.selenium.dev/ja/documentation/webdriver/browsers/firefox/
# #       collaborated with ChatGPT-4
# import subprocess
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from selenium.webdriver.firefox.options import Options
# from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
# from selenium.webdriver.common.by import By
#
#
# def fetch_text_with_selenium(url):
#     firefox_profile = FirefoxProfile()
#     firefox_profile.set_preference("javascript.enabled", True)
#     options = Options()
#     options.profile = firefox_profile
#     options.add_argument("--headless")
#     options.binary_location = "/usr/bin/firefox"
#     service = webdriver.FirefoxService(log_output=subprocess.STDOUT)
#     driver = webdriver.Firefox(options=options, service=service)
#
#     try:
#         # 指定されたURLにアクセス
#         driver.get(url)
#
#         # ページの全テキストを抽出
#         text = driver.find_element(by="tag name", value="body").text
#         return text
#     finally:
#         # ブラウザを閉じる
#         driver.quit()

In [ ]:
url = "http://example.com"
text = fetch_text_with_selenium(url)
print("-" * 120)
print(text)
print("-" * 120)

In [ ]:
url = "https://twitter.com/ai_database/status/1756147762822058066"
xpath = '//div[@id="react-root"]'
text = fetch_text_with_selenium(url, finds=[(By.XPATH, xpath, "Likes")])

In [ ]:
text

In [ ]:
# for cleaning of twitter message
def cleanup_text(text: str):
    if text is None:
        return None
    if "Conversation" not in text:
        # NOTE: possibly not x post
        return text

    do_skip = True
    filtered_lines = []
    for line in text.split("\n"):
        if line == "Conversation":
            do_skip = False
            continue
        if do_skip:
            continue
        filtered_lines.append(line)
    cleaned_text = "\n".join(filtered_lines)
    return cleaned_text


clean_text = cleanup_text(text)
print(clean_text)

In [ ]:
# NOTE: Generated by ChatGPT-4
import re


def extract_urls(text):
    # URLの正規表現パターン
    url_pattern = r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    urls = re.findall(url_pattern, text)
    return urls


# テスト用のテキスト
text = "この文には2つのURLが含まれています。https://www.example.com と http://www.test.com です。"

# URLを抽出
urls = extract_urls(text)
print(urls)

In [ ]:
def pickup_reference_text(url: str, finds: list = [], max_tries: int = 1) -> None | str:
    text = fetch_text_with_selenium(url, finds, max_tries=max_tries)
    text = cleanup_text(text)
    return text

# for X posts

In [ ]:
import time
import numpy as np
from selenium.webdriver.common.by import By


to_ref = {}
refs = []
for idx, row in df[["ツイート本文"]].iterrows():
    text = row.iloc[0]
    urls = extract_urls(text)
    if not urls:
        # url, ref_text
        refs.append((None, None))
        continue

    # NOTE: pickup the last url
    url = urls[-1]
    print(f"Processing [{url=}]", end="")

    finds = [
        (By.XPATH, '//div[@id="react-root"]', "Likes"),
        (By.TAG_NAME, "body", "\n"),
        (By.TAG_NAME, "body", "Copyright"),
    ]

    ref_text = pickup_reference_text(url, finds)
    to_ref[url] = ref_text
    refs.append((url, ref_text))

    # if idx > 0:
    #     break
    t = np.random.randint(3, 10)
    print(f" ... sleeping {t} secs")
    time.sleep(t)

In [ ]:
# ref_text = pickup_reference_text(url="https://t.co/Vrg02hwcE1", finds=finds)
# ref_text

In [ ]:
import pandas as pd


tsv_file = "../data/x_refs.tsv"
pd.DataFrame(refs, columns=["url", "text"]).to_csv(tsv_file, sep="\t", index=None)

In [ ]:
df = pd.read_csv(tsv_file, sep="\t", header=0)
df